In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, ConstantKernel as C
import tensorflow as tf

In [2]:
pip install scikit-learn

In [3]:
pwd

'C:\\Users\\Asus\\31_DL_Code_Basics'

In [4]:
import os
os.chdir("D:\Research work\Thesis")

In [5]:
# Load data from CSV files (replace file paths as needed)
array_PolyC = np.loadtxt("numpy_Polycrys_TRAIN.csv", delimiter=",", skiprows=1)
array_SingleC = np.loadtxt("numpy_SINGLEcrys.csv", delimiter=",", skiprows=1)

In [6]:
# Extract features and target values
array_PolyC_Fe = array_PolyC.T[0]  # Fe comp
array_PolyC_Ni = array_PolyC.T[1]  # Ni comp
array_PolyC_Tmax = array_PolyC.T[2]  # Tmax PolyCrystal

In [7]:
array_SingleC_Fe = array_SingleC.T[0]  # Fe comp
array_SingleC_Ni = array_SingleC.T[1]  # Ni comp
array_SingleC_Tmax = array_SingleC.T[2]  # Tmax SingleCrystal

In [8]:
# Combine features for low and high fidelity data
x_lf = np.array([array_SingleC_Fe, array_SingleC_Ni]).T  # (Fe, Ni) for SingleCrystal
y_lf = array_SingleC_Tmax.reshape((-1, 1))  # Reshape target to (, 1)

In [9]:
x_hf = np.array([array_PolyC_Fe, array_PolyC_Ni]).T  # (Fe, Ni) for PolyCrystal
y_hf = array_PolyC_Tmax.reshape((-1, 1))  # Reshape target to (, 1)

In [10]:
kernel_hf = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2)) + RationalQuadratic(0.1, 0.1, (1e-2, 1e2))
kernel_lf = C(0.1, (1e-3, 1e3)) * RBF(0.5, (1e-2, 1e2))

In [11]:
# Define kernels for high and low fidelity data
kernel_hf = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2)) + RationalQuadratic(0.1, 0.1, (1e-2, 1e2))
kernel_lf = C(0.1, (1e-3, 1e3)) * RBF(0.5, (1e-2, 1e2))

# Combine the kernels with a weighting factor (alpha) to create a multi-fidelity kernel
alpha = 0.2
kernel = (1 - alpha) * kernel_hf + alpha * kernel_lf

In [12]:
# Define kernels for high and low fidelity data
kernel_hf = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2)) + RationalQuadratic(0.1, 0.1, (1e-2, 1e2))
kernel_lf = C(0.1, (1e-3, 1e3)) * RBF(0.5, (1e-2, 1e2))

# Combine the kernels with a weighting factor (alpha) to create a multi-fidelity kernel
alpha = 0.2
kernel = (1 - alpha) * kernel_hf + alpha * kernel_lf

In [ ]:
# Create a Gaussian Process Regressor with the multi-fidelity kernel
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=8000000)

In [ ]:
X_combined = np.vstack((x_lf, x_hf))
y_combined = np.vstack((y_lf, y_hf))

gp.fit(X_combined, y_combined)
